<p style="background-color:#747173;font-family: 'Georgia';color:#FFFFFF;font-size:200%;text-align:center;border-radius:10px 10px;">Read Data</p>

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import chardet


with open(r"C:\Users\El-Wattaneya\Desktop\Task\AIA_cleaned.csv", 'r', encoding='utf-8') as file:
    aia_text = file.read()

with open(r"C:\Users\El-Wattaneya\Desktop\Task\FIDIC_cleaned.csv", 'r', encoding='utf-8') as file:
    fidic_text = file.read()

with open(r"C:\Users\El-Wattaneya\Desktop\Task\JCT_cleaned.csv", 'r', encoding='utf-8') as file:
    jct_text = file.read()

with open(r"C:\Users\El-Wattaneya\Desktop\Task\Nec_1_cleaned.csv", 'r', encoding='Windows-1252') as file:
    nec1_text = file.read()

<p style="background-color:#747173;font-family: 'Georgia';color:#FFFFFF;font-size:200%;text-align:center;border-radius:10px 10px;">Cleaning Data And Use BERT Model</p>

In [17]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
from threading import Thread

# تحميل نموذج BERT المتقدم
model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

def clean_text(text):
    if isinstance(text, str):
        # إزالة الأنماط مثل A.1.1
        text = re.sub(r'[A-Z]\.\d+\.\d+', '', text)
        # إزالة الرموز غير الضرورية
        text = re.sub(r'[^\w\s]', '', text)
        # إزالة الأرقام
        text = re.sub(r'\d+', '', text)
        # إزالة الفراغات الزائدة
        text = re.sub(r'\s+', ' ', text).strip()
        # تحويل النص إلى أحرف صغيرة
        text = text.lower()
        
        # إزالة الكلمات غير الضرورية
        non_informative_words = {"the", "and", "is", "in", "to", "of", "a", "that", "it", "with", "or", "from" , "he" , "she" ,"this" , "as" ,"by"}
        words = text.split()
        filtered_words = [word for word in words if word not in non_informative_words]
        text = ' '.join(filtered_words)
        
    return text




c:\Users\El-Wattaneya\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
from threading import Thread

# تحميل نموذج BERT المتقدم
model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

def clean_text(text):
    if isinstance(text, str):
        # إزالة الأنماط مثل A.1.1
        text = re.sub(r'[A-Z]\.\d+\.\d+', '', text)
        # إزالة الرموز غير الضرورية
        text = re.sub(r'[^\w\s]', '', text)
        # إزالة الأرقام
        text = re.sub(r'\d+', '', text)
        # إزالة الفراغات الزائدة
        text = re.sub(r'\s+', ' ', text).strip()
        # تحويل النص إلى أحرف صغيرة
        text = text.lower()
        
        # إزالة الكلمات غير الضرورية
        non_informative_words = {"the", "and", "is", "in", "to", "of", "a", "that", "it", "with", "or", "from","he", "she","this", "as","by",}
        words = text.split()
        filtered_words = [word for word in words if word not in non_informative_words]
        text = ' '.join(filtered_words)
        
    return text

def get_sentence_embeddings(sentences):
    return model.encode(sentences, convert_to_tensor=True)

def calculate_similarity(embeddings1, embeddings2):
    return util.pytorch_cos_sim(embeddings1, embeddings2)

def compare_files(file1_path, file2_path):
    df1 = pd.read_csv(file1_path, header=None, names=['sentence'])
    df2 = pd.read_csv(file2_path, header=None, names=['sentence'])

    sentences1 = df1['sentence'].apply(clean_text).tolist()
    sentences2 = df2['sentence'].apply(clean_text).tolist()

    embeddings1 = get_sentence_embeddings(sentences1)
    embeddings2 = get_sentence_embeddings(sentences2)

    similarities = calculate_similarity(embeddings1, embeddings2)

    results = []
    for i, sentence1 in enumerate(sentences1):
        max_similarity = similarities[i].max().item()
        best_match_index = similarities[i].argmax().item()
        best_match_sentence = sentences2[best_match_index]
        results.append({
            'sentence1': sentence1,
            'best_match_sentence': best_match_sentence,
            'similarity': max_similarity
        })

    # تصفية النتائج بناءً على حد أدنى للتشابه
    min_similarity_threshold = 0.7
    filtered_results = [result for result in results if result['similarity'] >= min_similarity_threshold]

    return filtered_results



<p style="background-color:#747173;font-family: 'Georgia';color:#FFFFFF;font-size:200%;text-align:center;border-radius:10px 10px;">GUI by Tinker</p>

In [19]:
from tkinter import filedialog, scrolledtext, messagebox
import tkinter as tk
import pandas as pd

def upload_file1():
    global file1_path
    file1_path = filedialog.askopenfilename(filetypes=[('CSV files', '*.csv')])
    if file1_path:
        file1_label.config(text=f"File 1: {file1_path.split('/')[-1]}")

def upload_file2():
    global file2_path
    file2_path = filedialog.askopenfilename(filetypes=[('CSV files', '*.csv')])
    if file2_path:
        file2_label.config(text=f"File 2: {file2_path.split('/')[-1]}")

def check_similarity():
    if not file1_path or not file2_path:
        messagebox.showerror("Error", "Please upload both files.")
        return

    result_display.config(state=tk.NORMAL)
    result_display.delete(1.0, tk.END)
    result_display.insert(tk.END, "Processing...\n")
    result_display.config(state=tk.DISABLED)
    
    threading.Thread(target=process_files).start()

def process_files():
    results = compare_files(file1_path, file2_path)
    result_display.config(state=tk.NORMAL)
    result_display.delete(1.0, tk.END)
    
    result_display.insert(tk.END, "The result =\n\n", 'blue')
    for result in results:
        result_display.insert(tk.END, "Sentence 1: ", 'blue')
        result_display.insert(tk.END, f"{result['sentence1']}\n")
        result_display.insert(tk.END, "Best Match Sentence: ", 'blue')
        result_display.insert(tk.END, f"{result['best_match_sentence']}\n")
        result_display.insert(tk.END, "Similarity: ", 'blue')
        result_display.insert(tk.END, f"{result['similarity']:.2f}\n\n")
    
    result_display.config(state=tk.DISABLED)


In [20]:
import threading

root = tk.Tk()
root.title("Sentence Similarity Checker")
root.geometry("800x600")

frame = tk.Frame(root)
frame.pack(pady=20)

upload_button1 = tk.Button(frame, text="Upload File 1", command=upload_file1)
upload_button1.grid(row=0, column=0, padx=10)

upload_button2 = tk.Button(frame, text="Upload File 2", command=upload_file2)
upload_button2.grid(row=0, column=1, padx=10)

file1_label = tk.Label(frame, text="File 1: Not selected")
file1_label.grid(row=1, column=0, padx=10, pady=10)

file2_label = tk.Label(frame, text="File 2: Not selected")
file2_label.grid(row=1, column=1, padx=10, pady=10)

check_button = tk.Button(root, text="Check Similarity", command=check_similarity)
check_button.pack(pady=20)

result_display = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=100, height=30, state=tk.DISABLED)
result_display.tag_config('blue', foreground='blue')
result_display.pack(padx=20, pady=20)

root.mainloop()
